In [2]:
from vina import Vina

In [12]:
v = Vina(sf_name='vina')

In [3]:
from openbabel import pybel
import os

In [3]:
import os
import subprocess

def pdb_to_pdbqt(pdb_file, pdbqt_file):
    """将 PDB 文件转换为 PDBQT 文件"""
    command = [
        '/root/mgltools_x86_64Linux2_1.5.7/bin/pythonsh',
        # '/root/mgltools_x86_64Linux2_1.5.7/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand.py',
        '/root/mgltools_x86_64Linux2_1.5.7/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor.py',
        '-r', pdb_file,
        '-o', pdbqt_file
    ]
    print(f"Running command: {' '.join(command)}")
    
    # 运行命令并捕获输出和错误信息
    result = subprocess.run(command, capture_output=True, text=True)
    
    # 打印详细的输出和错误信息
    print(f"Command output: {result.stdout}")
    print(f"Command error: {result.stderr}")
    
    if result.returncode != 0:
        raise RuntimeError(f"Error converting {pdb_file} to {pdbqt_file}: {result.stderr}")
    
    print(f'Wrote converted file to {pdbqt_file}')
    return pdbqt_file

if __name__ == '__main__':
    # 指定输入的 PDB 文件路径
    pdb_file = '/root/autodl-tmp/PP_generate_v1/data/PPDbench/1cjr/receptor.pdb'
    
    # 指定输出的 PDBQT 文件路径
    pdbqt_file = '/root/autodl-tmp/PP_generate_v1/data/PPDbench/1cjr/receptor.pdbqt'
    
    # 确保输出目录存在
    os.makedirs(os.path.dirname(pdbqt_file), exist_ok=True)
    
    # 检查输入文件是否存在
    if not os.path.exists(pdb_file):
        raise FileNotFoundError(f"Input file {pdb_file} does not exist.")
    
    # 检查输入文件是否可读
    if not os.access(pdb_file, os.R_OK):
        raise PermissionError(f"Input file {pdb_file} is not readable.")
    
    # 转换 PDB 文件为 PDBQT 文件
    try:
        pdb_to_pdbqt(pdb_file, pdbqt_file)
    except Exception as e:
        print(f"An error occurred: {e}")

Running command: /root/mgltools_x86_64Linux2_1.5.7/bin/pythonsh /root/mgltools_x86_64Linux2_1.5.7/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor.py -r /root/autodl-tmp/PP_generate_v1/data/PPDbench/1cjr/receptor.pdb -o /root/autodl-tmp/PP_generate_v1/data/PPDbench/1cjr/receptor.pdbqt
Command output: setting PYTHONHOME environment
receptor:B:HIS25 missing both hydrogens!
receptor:B:HIS82 missing both hydrogens!
receptor:B:HIS96 missing both hydrogens!
polar hydrogens missing from n-terminus of chain B

Command error: 
Wrote converted file to /root/autodl-tmp/PP_generate_v1/data/PPDbench/1cjr/receptor.pdbqt


In [19]:
#直接给pdbqt的string就行
root = '/home/zht/programs/PP_generate/data/ppbench2024/1a0m_A/'
peptide_file = os.path.join(root,'peptide.pdb')
receptor_file = os.path.join(root,'receptor.pdb')
mol = next(pybel.readfile('pdb',peptide_file))
mol.write('pdbqt','/home/zht/programs/PP_generate/data/ppbench2024/1a0m_A/peptide.pdbqt',overwrite=True)
# re_mol = next(pybel.readfile('pdb',receptor_file))
# pdbqt_str = re_mol.write('pdbqt', './re.pdbqt', overwrite=True)

In [3]:
from vina import Vina
re_pdbqt_file = '/root/autodl-tmp/PP_generate_v1/data/PPDbench/1cjr/receptor.pdbqt'
pep_pdbqt_file = '/root/autodl-tmp/PP_generate_v1/data/PPDbench/1cjr/peptide.pdbqt'
v = Vina()
v.set_receptor(re_pdbqt_file)
# v.set_ligand_from_file(pep_pdbqt_file)

In [ ]:
import os
from vina import Vina
from openbabel import pybel
from Bio import PDB
import numpy as np
import tempfile

In [4]:
v = Vina(sf_name='vina')

In [18]:
class MySelect(PDB.Select):
    '''预处理蛋白质---选择类'''
    def accept_residue(self, residue):
    # 过滤 保留主要的氨基酸残基
    # 只接受非水分子和非离子
        return residue.get_resname() not in ['HOH', 'Na', 'Cl', 'K', 'Mg', 'Ca']
        
#过滤函数    
def filter_structure_without_h2oNion(protein_structure,protein_name):
    '''预处理蛋白质---过滤水和离子'''
    filtered_protein_structure = PDB.Structure.Structure("filtered_"+protein_name)
    for model in protein_structure:
        new_model = PDB.Model.Model(model.id)
        for chain in model:
            new_chain = PDB.Chain.Chain(chain.id)
            for residue in chain:
                if MySelect().accept_residue(residue):
                    new_chain.add(residue.copy())  # 添加过滤后的残基
            if len(new_chain) > 0:
                new_model.add(new_chain)
        filtered_protein_structure.add(new_model)
    return filtered_protein_structure


def get_interaction_windows(threshold,filtered_receptor_structure,filtered_peptide_structure):
    '''获取窗口位置坐标'''
    #获取多肽原子位置
    peptide_atoms = [atom for model in filtered_peptide_structure for chain in model for residue in chain for atom in residue]
    peptide_coords = [atom.coord for atom in peptide_atoms]

     # 获取受体的残基 is_aa保证是氨基酸
    receptor_residues = [residue for model in filtered_receptor_structure for chain in model for residue in chain if PDB.is_aa(residue)]

    interaction_windows = []
    for residue in receptor_residues:
        for peptide_coord in peptide_coords:
            # 计算坐标的距离 
            distance = residue['CA'].coord - peptide_coord
            if np.linalg.norm(distance) <= threshold:
                # 如果在阈值内，保存受体残基坐标(CA为中心)
                interaction_windows.append(residue['CA'].coord)
                break  # 找到一个相互作用后，可以跳出循环
    return interaction_windows

In [5]:
#转换pdb文件为pdbqt 并返回已经加载受体和配体的v
def read_pdb_files_into_vina(peptide_path,receptor_path,interaction_windows=None):
    v = Vina(sf_name='vina')
    peptide_mol = next(pybel.readfile('pdb',peptide_path))
    receptor_mol = next(pybel.readfile('pdb',receptor_path))
    #手动创建临时文件
    temp_receptor_path = receptor_path.replace('.pdb', '_temp.pdbqt')
    receptor_mol.write('pdbqt', temp_receptor_path)
    #创建不成功
    if not os.path.exists(temp_receptor_path):
        print(f"Error: Temporary receptor file {temp_receptor_path} was not created.")
    v.set_receptor(temp_receptor_path)

    peptide_pdbqt_str = peptide_mol.write('pdbqt')
    v.set_ligand_from_string(peptide_pdbqt_str)

    # #设置搜索空间
    # box_size = []



In [7]:
import os
#直接给pdbqt的string就行
root = '/root/autodl-tmp/PP_generate_v1/data/PPDbench/1cjr/'
peptide_path = os.path.join(root,'peptide.pdb')
receptor_path = os.path.join(root,'receptor.pdb')

In [8]:
v = read_pdb_files_into_vina(peptide_path,receptor_path)

NameError: name 'pybel' is not defined